In [ ]:
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install numpy
!pip install qiskit-transpiler-service

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 39.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info.operators import Operator
from qiskit.compiler import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_transpiler_service.transpiler_service import TranspilerService
import numpy as np

In [ ]:
QiskitRuntimeService.save_account(channel="ibm_quantum", token="48ee0c8e4e7a0e4b01217f879484b394659c082adb4bda925ef14376f1a72f677e5393bc881f5dee268df994d2668deb8a5722fcefe4a6d10cd5fd7142a2bb53",
                                  set_as_default=True, overwrite=True)
service = QiskitRuntimeService()
#service = QiskitRuntimeService(channel = 'ibm_quantum', instance='ibm-q/open/main', token = '48ee0c8e4e7a0e4b01217f879484b394659c082adb4bda925ef14376f1a72f677e5393bc881f5dee268df994d2668deb8a5722fcefe4a6d10cd5fd7142a2bb53')
#backend = service.get_backend("simulator_mps")
#backend = AerSimulator(method='matrix_product_state')
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=100)
cloud_transpiler_service = TranspilerService(
    backend_name=backend.name,
    ai=False,
    optimization_level=3
)

qiskit_runtime_service.__init__:INFO:2024-04-25 16:30:04,087: Default instance: ibm-q/open/main


In [ ]:
def gn_gate_controlled(n):
    circ = QuantumCircuit(1)
    #Unitary matrix of G operator
    G = Operator([
        [np.sqrt(1./n), -np.sqrt(1 - 1./n)],
        [np.sqrt(1 - 1./n), np.sqrt(1./n)]])
    #convert into controlled gate
    circ.unitary(G, [0])
    gate = circ.to_gate().control(1)
    return gate

In [ ]:
def gn_gate_transpose_controlled(n):
    circ = QuantumCircuit(1)
    #Unitary matrix of G operator
    G = Operator([
        [np.sqrt(1./n), np.sqrt(1 - 1./n)],
        [-np.sqrt(1 - 1./n), np.sqrt(1./n)]])
    #convert into controlled gate
    circ.unitary(G, [0])
    gate = circ.to_gate().control(1)
    return gate

In [ ]:
def initialize(circ, row1, row2, row3, row4, row5, row6, row7, row8, ps):
    #set phase shift qubit to |-> by applying X and H gates
    circ.x(ps[0])
    circ.h(ps[0])

    # apply X gate into first qubits of each rows
    circ.x(row1[0])
    circ.x(row2[0])
    circ.x(row3[0])
    circ.x(row4[0])
    circ.x(row5[0])
    circ.x(row6[0])
    circ.x(row7[0])
    circ.x(row8[0])

    #uniform superposition (W state)
    circ = W(circ, row1, row2, row3, row4, row5, row6, row7, row8)

    return circ

In [ ]:
def W(circ, row1, row2, row3, row4, row5, row6, row7, row8):
    # controlled-G(n) gates for uniform superposition
    G8 = gn_gate_controlled(8)
    G7 = gn_gate_controlled(7)
    G6 = gn_gate_controlled(6)
    G5 = gn_gate_controlled(5)
    G4 = gn_gate_controlled(4)
    G3 = gn_gate_controlled(3)
    G2 = gn_gate_controlled(2)
    # apply controlled G(n) to 0, 1 quibits of each row
    # then apply CNOT to 1, 0
    # repeat with decremented n
    circ.append(G8, [row1[0], row1[1]])
    circ.append(G8, [row2[0], row2[1]])
    circ.append(G8, [row3[0], row3[1]])
    circ.append(G8, [row4[0], row4[1]])
    circ.append(G8, [row5[0], row5[1]])
    circ.append(G8, [row6[0], row6[1]])
    circ.append(G8, [row7[0], row7[1]])
    circ.append(G8, [row8[0], row8[1]])
    circ.cx(row1[1], row1[0])
    circ.cx(row2[1], row2[0])
    circ.cx(row3[1], row3[0])
    circ.cx(row4[1], row4[0])
    circ.cx(row5[1], row5[0])
    circ.cx(row6[1], row6[0])
    circ.cx(row7[1], row7[0])
    circ.cx(row8[1], row8[0])
    circ.append(G7, [row1[1], row1[2]])
    circ.append(G7, [row2[1], row2[2]])
    circ.append(G7, [row3[1], row3[2]])
    circ.append(G7, [row4[1], row4[2]])
    circ.append(G7, [row5[1], row5[2]])
    circ.append(G7, [row6[1], row6[2]])
    circ.append(G7, [row7[1], row7[2]])
    circ.append(G7, [row8[1], row8[2]])
    circ.cx(row1[2], row1[1])
    circ.cx(row2[2], row2[1])
    circ.cx(row3[2], row3[1])
    circ.cx(row4[2], row4[1])
    circ.cx(row5[2], row5[1])
    circ.cx(row6[2], row6[1])
    circ.cx(row7[2], row7[1])
    circ.cx(row8[2], row8[1])
    circ.append(G6, [row1[2], row1[3]])
    circ.append(G6, [row2[2], row2[3]])
    circ.append(G6, [row3[2], row3[3]])
    circ.append(G6, [row4[2], row4[3]])
    circ.append(G6, [row5[2], row5[3]])
    circ.append(G6, [row6[2], row6[3]])
    circ.append(G6, [row7[2], row7[3]])
    circ.append(G6, [row8[2], row8[3]])
    circ.cx(row1[3], row1[2])
    circ.cx(row2[3], row2[2])
    circ.cx(row3[3], row3[2])
    circ.cx(row4[3], row4[2])
    circ.cx(row5[3], row5[2])
    circ.cx(row6[3], row6[2])
    circ.cx(row7[3], row7[2])
    circ.cx(row8[3], row8[2])
    circ.append(G5, [row1[3], row1[4]])
    circ.append(G5, [row2[3], row2[4]])
    circ.append(G5, [row3[3], row3[4]])
    circ.append(G5, [row4[3], row4[4]])
    circ.append(G5, [row5[3], row5[4]])
    circ.append(G5, [row6[3], row6[4]])
    circ.append(G5, [row7[3], row7[4]])
    circ.append(G5, [row8[3], row8[4]])
    circ.cx(row1[4], row1[3])
    circ.cx(row2[4], row2[3])
    circ.cx(row3[4], row3[3])
    circ.cx(row4[4], row4[3])
    circ.cx(row5[4], row5[3])
    circ.cx(row6[4], row6[3])
    circ.cx(row7[4], row7[3])
    circ.cx(row8[4], row8[3])
    circ.append(G4, [row1[4], row1[5]])
    circ.append(G4, [row2[4], row2[5]])
    circ.append(G4, [row3[4], row3[5]])
    circ.append(G4, [row4[4], row4[5]])
    circ.append(G4, [row5[4], row5[5]])
    circ.append(G4, [row6[4], row6[5]])
    circ.append(G4, [row7[4], row7[5]])
    circ.append(G4, [row8[4], row8[5]])
    circ.cx(row1[5], row1[4])
    circ.cx(row2[5], row2[4])
    circ.cx(row3[5], row3[4])
    circ.cx(row4[5], row4[4])
    circ.cx(row5[5], row5[4])
    circ.cx(row6[5], row6[4])
    circ.cx(row7[5], row7[4])
    circ.cx(row8[5], row8[4])
    circ.append(G3, [row1[5], row1[6]])
    circ.append(G3, [row2[5], row2[6]])
    circ.append(G3, [row3[5], row3[6]])
    circ.append(G3, [row4[5], row4[6]])
    circ.append(G3, [row5[5], row5[6]])
    circ.append(G3, [row6[5], row6[6]])
    circ.append(G3, [row7[5], row7[6]])
    circ.append(G3, [row8[5], row8[6]])
    circ.cx(row1[6], row1[5])
    circ.cx(row2[6], row2[5])
    circ.cx(row3[6], row3[5])
    circ.cx(row4[6], row4[5])
    circ.cx(row5[6], row5[5])
    circ.cx(row6[6], row6[5])
    circ.cx(row7[6], row7[5])
    circ.cx(row8[6], row8[5])
    circ.append(G2, [row1[6], row1[7]])
    circ.append(G2, [row2[6], row2[7]])
    circ.append(G2, [row3[6], row3[7]])
    circ.append(G2, [row4[6], row4[7]])
    circ.append(G2, [row5[6], row5[7]])
    circ.append(G2, [row6[6], row6[7]])
    circ.append(G2, [row7[6], row7[7]])
    circ.append(G2, [row8[6], row8[7]])
    circ.cx(row1[7], row1[6])
    circ.cx(row2[7], row2[6])
    circ.cx(row3[7], row3[6])
    circ.cx(row4[7], row4[6])
    circ.cx(row5[7], row5[6])
    circ.cx(row6[7], row6[6])
    circ.cx(row7[7], row7[6])
    circ.cx(row8[7], row8[6])

    return circ

In [ ]:
def W_inv(circ, row1, row2, row3, row4, row5, row6, row7, row8):
    G8 = gn_gate_transpose_controlled(8)
    G7 = gn_gate_transpose_controlled(7)
    G6 = gn_gate_transpose_controlled(6)
    G5 = gn_gate_transpose_controlled(5)
    G4 = gn_gate_transpose_controlled(4)
    G3 = gn_gate_transpose_controlled(3)
    G2 = gn_gate_transpose_controlled(2)

    circ.cx(row1[7], row1[6])
    circ.cx(row2[7], row2[6])
    circ.cx(row3[7], row3[6])
    circ.cx(row4[7], row4[6])
    circ.cx(row5[7], row5[6])
    circ.cx(row6[7], row6[6])
    circ.cx(row7[7], row7[6])
    circ.cx(row8[7], row8[6])
    circ.append(G2, [row1[6], row1[7]])
    circ.append(G2, [row2[6], row2[7]])
    circ.append(G2, [row3[6], row3[7]])
    circ.append(G2, [row4[6], row4[7]])
    circ.append(G2, [row5[6], row5[7]])
    circ.append(G2, [row6[6], row6[7]])
    circ.append(G2, [row7[6], row7[7]])
    circ.append(G2, [row8[6], row8[7]])
    circ.cx(row1[6], row1[5])
    circ.cx(row2[6], row2[5])
    circ.cx(row3[6], row3[5])
    circ.cx(row4[6], row4[5])
    circ.cx(row5[6], row5[5])
    circ.cx(row6[6], row6[5])
    circ.cx(row7[6], row7[5])
    circ.cx(row8[6], row8[5])
    circ.append(G3, [row1[5], row1[6]])
    circ.append(G3, [row2[5], row2[6]])
    circ.append(G3, [row3[5], row3[6]])
    circ.append(G3, [row4[5], row4[6]])
    circ.append(G3, [row5[5], row5[6]])
    circ.append(G3, [row6[5], row6[6]])
    circ.append(G3, [row7[5], row7[6]])
    circ.append(G3, [row8[5], row8[6]])
    circ.cx(row1[5], row1[4])
    circ.cx(row2[5], row2[4])
    circ.cx(row3[5], row3[4])
    circ.cx(row4[5], row4[4])
    circ.cx(row5[5], row5[4])
    circ.cx(row6[5], row6[4])
    circ.cx(row7[5], row7[4])
    circ.cx(row8[5], row8[4])
    circ.append(G4, [row1[4], row1[5]])
    circ.append(G4, [row2[4], row2[5]])
    circ.append(G4, [row3[4], row3[5]])
    circ.append(G4, [row4[4], row4[5]])
    circ.append(G4, [row5[4], row5[5]])
    circ.append(G4, [row6[4], row6[5]])
    circ.append(G4, [row7[4], row7[5]])
    circ.append(G4, [row8[4], row8[5]])
    circ.cx(row1[4], row1[3])
    circ.cx(row2[4], row2[3])
    circ.cx(row3[4], row3[3])
    circ.cx(row4[4], row4[3])
    circ.cx(row5[4], row5[3])
    circ.cx(row6[4], row6[3])
    circ.cx(row7[4], row7[3])
    circ.cx(row8[4], row8[3])
    circ.append(G5, [row1[3], row1[4]])
    circ.append(G5, [row2[3], row2[4]])
    circ.append(G5, [row3[3], row3[4]])
    circ.append(G5, [row4[3], row4[4]])
    circ.append(G5, [row5[3], row5[4]])
    circ.append(G5, [row6[3], row6[4]])
    circ.append(G5, [row7[3], row7[4]])
    circ.append(G5, [row8[3], row8[4]])
    circ.cx(row1[3], row1[2])
    circ.cx(row2[3], row2[2])
    circ.cx(row3[3], row3[2])
    circ.cx(row4[3], row4[2])
    circ.cx(row5[3], row5[2])
    circ.cx(row6[3], row6[2])
    circ.cx(row7[3], row7[2])
    circ.cx(row8[3], row8[2])
    circ.append(G6, [row1[2], row1[3]])
    circ.append(G6, [row2[2], row2[3]])
    circ.append(G6, [row3[2], row3[3]])
    circ.append(G6, [row4[2], row4[3]])
    circ.append(G6, [row5[2], row5[3]])
    circ.append(G6, [row6[2], row6[3]])
    circ.append(G6, [row7[2], row7[3]])
    circ.append(G6, [row8[2], row8[3]])
    circ.cx(row1[2], row1[1])
    circ.cx(row2[2], row2[1])
    circ.cx(row3[2], row3[1])
    circ.cx(row4[2], row4[1])
    circ.cx(row5[2], row5[1])
    circ.cx(row6[2], row6[1])
    circ.cx(row7[2], row7[1])
    circ.cx(row8[2], row8[1])
    circ.append(G7, [row1[1], row1[2]])
    circ.append(G7, [row2[1], row2[2]])
    circ.append(G7, [row3[1], row3[2]])
    circ.append(G7, [row4[1], row4[2]])
    circ.append(G7, [row5[1], row5[2]])
    circ.append(G7, [row6[1], row6[2]])
    circ.append(G7, [row7[1], row7[2]])
    circ.append(G7, [row8[1], row8[2]])
    circ.cx(row1[1], row1[0])
    circ.cx(row2[1], row2[0])
    circ.cx(row3[1], row3[0])
    circ.cx(row4[1], row4[0])
    circ.cx(row5[1], row5[0])
    circ.cx(row6[1], row6[0])
    circ.cx(row7[1], row7[0])
    circ.cx(row8[1], row8[0])
    circ.append(G8, [row1[0], row1[1]])
    circ.append(G8, [row2[0], row2[1]])
    circ.append(G8, [row3[0], row3[1]])
    circ.append(G8, [row4[0], row4[1]])
    circ.append(G8, [row5[0], row5[1]])
    circ.append(G8, [row6[0], row6[1]])
    circ.append(G8, [row7[0], row7[1]])
    circ.append(G8, [row8[0], row8[1]])

    return circ

In [ ]:
def col_cond(circ, row1, row2, row3, row4, row5, row6, row7, row8, col_result):
    # qubit 0 is |1> for each rows
    for i in range(7):
        circ.x(row1[i + 1])
        circ.x(row2[i + 1])
        circ.x(row3[i + 1])
        circ.x(row4[i + 1])
        circ.x(row5[i + 1])
        circ.x(row6[i + 1])
        circ.x(row7[i + 1])
        circ.x(row8[i + 1])
    # AND using toffoli gate
    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])
    # qubit 1 is |1> for each rows
    # repeat until qubit 7
    circ.x(row1[0])
    circ.x(row1[1])
    circ.x(row2[0])
    circ.x(row2[1])
    circ.x(row3[0])
    circ.x(row3[1])
    circ.x(row4[0])
    circ.x(row4[1])
    circ.x(row5[0])
    circ.x(row5[1])
    circ.x(row6[0])
    circ.x(row6[1])
    circ.x(row7[0])
    circ.x(row8[1])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    circ.x(row1[1])
    circ.x(row1[2])
    circ.x(row2[1])
    circ.x(row2[2])
    circ.x(row3[1])
    circ.x(row3[2])
    circ.x(row4[1])
    circ.x(row4[2])
    circ.x(row5[1])
    circ.x(row5[2])
    circ.x(row6[1])
    circ.x(row6[2])
    circ.x(row7[1])
    circ.x(row8[2])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    circ.x(row1[2])
    circ.x(row1[3])
    circ.x(row2[2])
    circ.x(row2[3])
    circ.x(row3[2])
    circ.x(row3[3])
    circ.x(row4[2])
    circ.x(row4[3])
    circ.x(row5[2])
    circ.x(row5[3])
    circ.x(row6[2])
    circ.x(row6[3])
    circ.x(row7[2])
    circ.x(row8[3])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    circ.x(row1[3])
    circ.x(row1[4])
    circ.x(row2[3])
    circ.x(row2[4])
    circ.x(row3[3])
    circ.x(row3[4])
    circ.x(row4[3])
    circ.x(row4[4])
    circ.x(row5[3])
    circ.x(row5[4])
    circ.x(row6[3])
    circ.x(row6[4])
    circ.x(row7[3])
    circ.x(row8[4])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    circ.x(row1[4])
    circ.x(row1[5])
    circ.x(row2[4])
    circ.x(row2[5])
    circ.x(row3[4])
    circ.x(row3[5])
    circ.x(row4[4])
    circ.x(row4[5])
    circ.x(row5[4])
    circ.x(row5[5])
    circ.x(row6[4])
    circ.x(row6[5])
    circ.x(row7[4])
    circ.x(row8[5])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    circ.x(row1[5])
    circ.x(row1[6])
    circ.x(row2[5])
    circ.x(row2[6])
    circ.x(row3[5])
    circ.x(row3[6])
    circ.x(row4[5])
    circ.x(row4[6])
    circ.x(row5[5])
    circ.x(row5[6])
    circ.x(row6[5])
    circ.x(row6[6])
    circ.x(row7[5])
    circ.x(row8[6])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    circ.x(row1[6])
    circ.x(row1[7])
    circ.x(row2[6])
    circ.x(row2[7])
    circ.x(row3[6])
    circ.x(row3[7])
    circ.x(row4[6])
    circ.x(row4[7])
    circ.x(row5[6])
    circ.x(row5[7])
    circ.x(row6[6])
    circ.x(row6[7])
    circ.x(row7[6])
    circ.x(row8[7])

    circ.mcx([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7]], col_result[3])
    circ.mcx([row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7]], col_result[4])
    circ.mcx([row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7]], col_result[5])
    circ.mcx([row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7]], col_result[6])
    circ.mcx([row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]], col_result[7])

    #reset all qubits into |1>
    for i in range(7):
        circ.x(row1[i])
        circ.x(row2[i])
        circ.x(row3[i])
        circ.x(row4[i])
        circ.x(row5[i])
        circ.x(row6[i])
        circ.x(row7[i])
        circ.x(row8[i])

    return circ

In [ ]:
def dg_cond(circ, row1, row2, row3, row4, row5, row6, row7, row8,
                  dg_result_1, dg_result_2, dg_result_3, dg_result_4):
    # eq3
    # n=3
    circ.ccx(row1[1], row2[0], dg_result_1[0])
    circ.x(dg_result_1[0])

    # n=4
    circ.x(row2[1])
    circ.x(row3[0])

    circ.mcx([row1[2], row2[1], row3[0]], dg_result_1[1])

    circ.x(row1[2])
    circ.x(row2[1])

    circ.mcx([row1[2], row2[1], row3[0]], dg_result_1[1])

    circ.x(row2[1])

    circ.mcx([row1[2], row2[1], row3[0]], dg_result_1[1])

    circ.x(row1[2])
    circ.x(row2[1])
    circ.x(row3[0])

    # n=5
    circ.x(row2[2])
    circ.x(row3[1])
    circ.x(row4[0])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row1[3])
    circ.x(row2[2])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row2[2])
    circ.x(row3[1])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row3[1])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row1[3])
    circ.x(row2[2])
    circ.x(row3[1])
    circ.x(row4[0])

    # n=6
    circ.x(row2[3])
    circ.x(row3[2])
    circ.x(row4[1])
    circ.x(row5[0])

    circ.mcx([row1[4], row2[3], row3[2], row4[1], row5[0]], dg_result_1[3])

    circ.x(row1[4])
    circ.x(row2[3])

    circ.mcx([row1[4], row2[3], row3[2], row4[1], row5[0]], dg_result_1[3])

    circ.x(row2[3])
    circ.x(row3[2])

    circ.mcx([row1[4], row2[3], row3[2], row4[1], row5[0]], dg_result_1[3])

    circ.x(row3[2])
    circ.x(row4[1])

    circ.mcx([row1[4], row2[3], row3[2], row4[1], row5[0]], dg_result_1[3])

    circ.x(row4[1])

    circ.mcx([row1[4], row2[3], row3[2], row4[1], row5[0]], dg_result_1[3])

    circ.x(row1[4])
    circ.x(row2[3])
    circ.x(row3[2])
    circ.x(row4[1])
    circ.x(row5[0])

    # n=7
    circ.x(row2[4])
    circ.x(row3[3])
    circ.x(row4[2])
    circ.x(row5[1])
    circ.x(row6[0])

    circ.mcx([row1[5], row2[4], row3[3], row4[2], row5[1], row6[0]], dg_result_1[4])

    circ.x(row1[5])
    circ.x(row2[4])

    circ.mcx([row1[5], row2[4], row3[3], row4[2], row5[1], row6[0]], dg_result_1[4])

    circ.x(row2[4])
    circ.x(row3[3])

    circ.mcx([row1[5], row2[4], row3[3], row4[2], row5[1], row6[0]], dg_result_1[4])

    circ.x(row3[3])
    circ.x(row4[2])

    circ.mcx([row1[5], row2[4], row3[3], row4[2], row5[1], row6[0]], dg_result_1[4])

    circ.x(row4[2])
    circ.x(row5[1])

    circ.mcx([row1[5], row2[4], row3[3], row4[2], row5[1], row6[0]], dg_result_1[4])

    circ.x(row5[1])

    circ.mcx([row1[5], row2[4], row3[3], row4[2], row5[1], row6[0]], dg_result_1[4])

    circ.x(row1[5])
    circ.x(row2[4])
    circ.x(row3[3])
    circ.x(row4[2])
    circ.x(row5[1])
    circ.x(row6[0])

    # n=8
    circ.x(row2[5])
    circ.x(row3[4])
    circ.x(row4[3])
    circ.x(row5[2])
    circ.x(row6[1])
    circ.x(row7[0])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row1[6])
    circ.x(row2[5])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row2[5])
    circ.x(row3[4])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row3[4])
    circ.x(row4[3])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row4[3])
    circ.x(row5[2])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row5[2])
    circ.x(row6[1])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row6[1])

    circ.mcx([row1[6], row2[5], row3[4], row4[3], row5[2], row6[1], row7[0]], dg_result_1[5])

    circ.x(row1[6])
    circ.x(row2[5])
    circ.x(row3[4])
    circ.x(row4[3])
    circ.x(row5[2])
    circ.x(row6[1])
    circ.x(row7[0])

    # n=9
    circ.x(row2[6])
    circ.x(row3[5])
    circ.x(row4[4])
    circ.x(row5[3])
    circ.x(row6[2])
    circ.x(row7[1])
    circ.x(row8[0])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row1[7])
    circ.x(row2[6])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row2[6])
    circ.x(row3[5])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row3[5])
    circ.x(row4[4])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row4[4])
    circ.x(row5[3])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row5[3])
    circ.x(row6[2])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row6[2])
    circ.x(row7[1])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row7[1])

    circ.mcx([row1[7], row2[6], row3[5], row4[4], row5[3], row6[2], row7[1], row8[0]], dg_result_1[6])

    circ.x(row1[7])
    circ.x(row2[6])
    circ.x(row3[5])
    circ.x(row4[4])
    circ.x(row5[3])
    circ.x(row6[2])
    circ.x(row7[1])
    circ.x(row8[0])

    #-------------------------------------------------------------------------------------------------
    # eq4
    # n=2
    circ.x(row3[6])
    circ.x(row4[5])
    circ.x(row5[4])
    circ.x(row6[3])
    circ.x(row7[2])
    circ.x(row8[1])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row2[7])
    circ.x(row3[6])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row3[6])
    circ.x(row4[5])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row4[5])
    circ.x(row5[4])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row5[4])
    circ.x(row6[3])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row6[3])
    circ.x(row7[2])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row7[2])

    circ.mcx([row2[7], row3[6], row4[5], row5[4], row6[3], row7[2], row8[1]], dg_result_2[0])

    circ.x(row2[7])
    circ.x(row3[6])
    circ.x(row4[5])
    circ.x(row5[4])
    circ.x(row6[3])
    circ.x(row7[2])
    circ.x(row8[1])

    # n=3
    circ.x(row4[6])
    circ.x(row5[5])
    circ.x(row6[4])
    circ.x(row7[3])
    circ.x(row8[2])

    circ.mcx([row3[7], row4[6], row5[5], row6[4], row7[3], row8[2]], dg_result_2[1])

    circ.x(row3[7])
    circ.x(row4[6])

    circ.mcx([row3[7], row4[6], row5[5], row6[4], row7[3], row8[2]], dg_result_2[1])

    circ.x(row4[6])
    circ.x(row5[5])

    circ.mcx([row3[7], row4[6], row5[5], row6[4], row7[3], row8[2]], dg_result_2[1])

    circ.x(row5[5])
    circ.x(row6[4])

    circ.mcx([row3[7], row4[6], row5[5], row6[4], row7[3], row8[2]], dg_result_2[1])

    circ.x(row6[4])
    circ.x(row7[3])

    circ.mcx([row3[7], row4[6], row5[5], row6[4], row7[3], row8[2]], dg_result_2[1])

    circ.x(row7[3])

    circ.mcx([row3[7], row4[6], row5[5], row6[4], row7[3], row8[2]], dg_result_2[1])

    circ.x(row3[7])
    circ.x(row4[6])
    circ.x(row5[5])
    circ.x(row6[4])
    circ.x(row7[3])
    circ.x(row8[2])

    # n=4
    circ.x(row5[6])
    circ.x(row6[5])
    circ.x(row7[4])
    circ.x(row8[3])

    circ.mcx([row4[7], row5[6], row6[5], row7[4], row8[3]], dg_result_2[2])

    circ.x(row4[7])
    circ.x(row5[6])

    circ.mcx([row4[7], row5[6], row6[5], row7[4], row8[3]], dg_result_2[2])

    circ.x(row5[6])
    circ.x(row6[5])

    circ.mcx([row4[7], row5[6], row6[5], row7[4], row8[3]], dg_result_2[2])

    circ.x(row6[5])
    circ.x(row7[4])

    circ.mcx([row4[7], row5[6], row6[5], row7[4], row8[3]], dg_result_2[2])

    circ.x(row7[4])

    circ.mcx([row4[7], row5[6], row6[5], row7[4], row8[3]], dg_result_2[2])

    circ.x(row4[7])
    circ.x(row5[6])
    circ.x(row6[5])
    circ.x(row7[4])
    circ.x(row8[3])

    # n=5
    circ.x(row6[6])
    circ.x(row7[5])
    circ.x(row8[4])

    circ.mcx([row5[7], row6[6], row7[5], row8[4]], dg_result_2[3])

    circ.x(row5[7])
    circ.x(row6[6])

    circ.mcx([row5[7], row6[6], row7[5], row8[4]], dg_result_2[3])

    circ.x(row6[6])
    circ.x(row7[5])

    circ.mcx([row5[7], row6[6], row7[5], row8[4]], dg_result_2[3])

    circ.x(row7[5])

    circ.mcx([row5[7], row6[6], row7[5], row8[4]], dg_result_2[3])

    circ.x(row5[7])
    circ.x(row6[6])
    circ.x(row7[5])
    circ.x(row8[4])

    # n=6
    circ.x(row7[6])
    circ.x(row8[5])

    circ.mcx([row6[7], row7[6], row8[5]], dg_result_2[4])

    circ.x(row6[7])
    circ.x(row7[6])

    circ.mcx([row6[7], row7[6], row8[5]], dg_result_2[4])

    circ.x(row7[6])

    circ.mcx([row6[7], row7[6], row8[5]], dg_result_2[4])

    circ.x(row6[7])
    circ.x(row7[6])
    circ.x(row8[5])

    # n = 7
    circ.ccx(row7[7], row8[6], dg_result_2[5])
    circ.x(dg_result_2[5])

    #-------------------------------------------------------------------------------------------------
    # eq5
    # n=0
    circ.x(row2[1])
    circ.x(row3[2])
    circ.x(row4[3])
    circ.x(row5[4])
    circ.x(row6[5])
    circ.x(row7[6])
    circ.x(row8[7])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row1[0])
    circ.x(row2[1])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row2[1])
    circ.x(row3[2])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row3[2])
    circ.x(row4[3])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row4[3])
    circ.x(row5[4])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row5[4])
    circ.x(row6[5])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row6[5])
    circ.x(row7[6])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row7[6])

    circ.mcx([row1[0], row2[1], row3[2], row4[3], row5[4], row6[5], row7[6], row8[7]], dg_result_3[0])

    circ.x(row1[0])
    circ.x(row2[1])
    circ.x(row3[2])
    circ.x(row4[3])
    circ.x(row5[4])
    circ.x(row6[5])
    circ.x(row7[6])
    circ.x(row8[7])

    # n=1
    circ.x(row2[2])
    circ.x(row3[3])
    circ.x(row4[4])
    circ.x(row5[5])
    circ.x(row6[6])
    circ.x(row7[7])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row1[1])
    circ.x(row2[2])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row2[2])
    circ.x(row3[3])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row3[3])
    circ.x(row4[4])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row4[4])
    circ.x(row5[5])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row5[5])
    circ.x(row6[6])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row6[6])

    circ.mcx([row1[1], row2[2], row3[3], row4[4], row5[5], row6[6], row7[7]], dg_result_3[1])

    circ.x(row1[1])
    circ.x(row2[2])
    circ.x(row3[3])
    circ.x(row4[4])
    circ.x(row5[5])
    circ.x(row6[6])
    circ.x(row7[7])

    # n=2
    circ.x(row2[3])
    circ.x(row3[4])
    circ.x(row4[5])
    circ.x(row5[6])
    circ.x(row6[7])

    circ.mcx([row1[2], row2[3], row3[4], row4[5], row5[6], row6[7]], dg_result_3[2])

    circ.x(row1[2])
    circ.x(row2[3])

    circ.mcx([row1[2], row2[3], row3[4], row4[5], row5[6], row6[7]], dg_result_3[2])

    circ.x(row2[3])
    circ.x(row3[4])

    circ.mcx([row1[2], row2[3], row3[4], row4[5], row5[6], row6[7]], dg_result_3[2])

    circ.x(row3[4])
    circ.x(row4[5])

    circ.mcx([row1[2], row2[3], row3[4], row4[5], row5[6], row6[7]], dg_result_3[2])

    circ.x(row4[5])
    circ.x(row5[6])

    circ.mcx([row1[2], row2[3], row3[4], row4[5], row5[6], row6[7]], dg_result_3[2])

    circ.x(row5[6])

    circ.mcx([row1[2], row2[3], row3[4], row4[5], row5[6], row6[7]], dg_result_3[2])

    circ.x(row1[2])
    circ.x(row2[3])
    circ.x(row3[4])
    circ.x(row4[5])
    circ.x(row5[6])
    circ.x(row6[7])

    # n=3
    circ.x(row2[4])
    circ.x(row3[5])
    circ.x(row4[6])
    circ.x(row5[7])

    circ.mcx([row1[3], row2[4], row3[5], row4[6], row5[7]], dg_result_3[3])

    circ.x(row1[3])
    circ.x(row2[4])

    circ.mcx([row1[3], row2[4], row3[5], row4[6], row5[7]], dg_result_3[3])

    circ.x(row2[4])
    circ.x(row3[5])

    circ.mcx([row1[3], row2[4], row3[5], row4[6], row5[7]], dg_result_3[3])

    circ.x(row3[5])
    circ.x(row4[6])

    circ.mcx([row1[3], row2[4], row3[5], row4[6], row5[7]], dg_result_3[3])

    circ.x(row4[6])

    circ.mcx([row1[3], row2[4], row3[5], row4[6], row5[7]], dg_result_3[3])

    circ.x(row1[3])
    circ.x(row2[4])
    circ.x(row3[5])
    circ.x(row4[6])
    circ.x(row5[7])

    # n=4
    circ.x(row2[5])
    circ.x(row3[6])
    circ.x(row4[7])

    circ.mcx([row1[4], row2[5], row3[6], row4[7]], dg_result_3[4])

    circ.x(row1[4])
    circ.x(row2[5])

    circ.mcx([row1[4], row2[5], row3[6], row4[7]], dg_result_3[4])

    circ.x(row2[5])
    circ.x(row3[6])

    circ.mcx([row1[4], row2[5], row3[6], row4[7]], dg_result_3[4])

    circ.x(row3[6])

    circ.mcx([row1[4], row2[5], row3[6], row4[7]], dg_result_3[4])

    circ.x(row1[4])
    circ.x(row2[5])
    circ.x(row3[6])
    circ.x(row4[7])

    # n=5
    circ.x(row2[6])
    circ.x(row3[7])

    circ.mcx([row1[5], row2[6], row3[7]], dg_result_3[5])

    circ.x(row1[5])
    circ.x(row2[6])

    circ.mcx([row1[5], row2[6], row3[7]], dg_result_3[5])

    circ.x(row2[6])

    circ.mcx([row1[5], row2[6], row3[7]], dg_result_3[5])

    circ.x(row1[5])
    circ.x(row2[6])
    circ.x(row3[7])

    # n=6
    circ.ccx(row1[6], row2[7], dg_result_3[6])
    circ.x(dg_result_3[6])

    #-------------------------------------------------------------------------------------------------
    # eq6
    # n=1
    circ.x(row3[1])
    circ.x(row4[2])
    circ.x(row5[3])
    circ.x(row6[4])
    circ.x(row7[5])
    circ.x(row8[6])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row2[0])
    circ.x(row3[1])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row3[1])
    circ.x(row4[2])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row4[2])
    circ.x(row5[3])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row5[3])
    circ.x(row6[4])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row6[4])
    circ.x(row7[5])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row7[5])

    circ.mcx([row2[0], row3[1], row4[2], row5[3], row6[4], row7[5], row8[6]], dg_result_4[0])

    circ.x(row2[0])
    circ.x(row3[1])
    circ.x(row4[2])
    circ.x(row5[3])
    circ.x(row6[4])
    circ.x(row7[5])
    circ.x(row8[6])

    # n=2
    circ.x(row4[1])
    circ.x(row5[2])
    circ.x(row6[3])
    circ.x(row7[4])
    circ.x(row8[5])

    circ.mcx([row3[0], row4[1], row5[2], row6[3], row7[4], row8[5]], dg_result_4[1])

    circ.x(row3[0])
    circ.x(row4[1])

    circ.mcx([row3[0], row4[1], row5[2], row6[3], row7[4], row8[5]], dg_result_4[1])

    circ.x(row4[1])
    circ.x(row5[2])

    circ.mcx([row3[0], row4[1], row5[2], row6[3], row7[4], row8[5]], dg_result_4[1])

    circ.x(row5[2])
    circ.x(row6[3])

    circ.mcx([row3[0], row4[1], row5[2], row6[3], row7[4], row8[5]], dg_result_4[1])

    circ.x(row6[3])
    circ.x(row7[4])

    circ.mcx([row3[0], row4[1], row5[2], row6[3], row7[4], row8[5]], dg_result_4[1])

    circ.x(row7[4])

    circ.mcx([row3[0], row4[1], row5[2], row6[3], row7[4], row8[5]], dg_result_4[1])

    circ.x(row3[0])
    circ.x(row4[1])
    circ.x(row5[2])
    circ.x(row6[3])
    circ.x(row7[4])
    circ.x(row8[5])

    # n=3
    circ.x(row5[1])
    circ.x(row6[2])
    circ.x(row7[3])
    circ.x(row8[4])

    circ.mcx([row4[0], row5[1], row6[2], row7[3], row8[4]], dg_result_4[2])

    circ.x(row4[0])
    circ.x(row5[1])

    circ.mcx([row4[0], row5[1], row6[2], row7[3], row8[4]], dg_result_4[2])

    circ.x(row5[1])
    circ.x(row6[2])

    circ.mcx([row4[0], row5[1], row6[2], row7[3], row8[4]], dg_result_4[2])

    circ.x(row6[2])
    circ.x(row7[3])

    circ.mcx([row4[0], row5[1], row6[2], row7[3], row8[4]], dg_result_4[2])

    circ.x(row7[3])

    circ.mcx([row4[0], row5[1], row6[2], row7[3], row8[4]], dg_result_4[2])

    circ.x(row4[0])
    circ.x(row5[1])
    circ.x(row6[2])
    circ.x(row7[3])
    circ.x(row8[4])

    # n=4
    circ.x(row6[1])
    circ.x(row7[2])
    circ.x(row8[3])

    circ.mcx([row5[0], row6[1], row7[2], row8[3]], dg_result_4[3])

    circ.x(row5[0])
    circ.x(row6[1])

    circ.mcx([row5[0], row6[1], row7[2], row8[3]], dg_result_4[3])

    circ.x(row6[1])
    circ.x(row7[2])

    circ.mcx([row5[0], row6[1], row7[2], row8[3]], dg_result_4[3])

    circ.x(row7[2])

    circ.mcx([row5[0], row6[1], row7[2], row8[3]], dg_result_4[3])

    circ.x(row5[0])
    circ.x(row6[1])
    circ.x(row7[2])
    circ.x(row8[3])

    # n=5
    circ.x(row7[1])
    circ.x(row8[2])

    circ.mcx([row6[0], row7[1], row8[2]], dg_result_4[4])

    circ.x(row6[0])
    circ.x(row7[1])

    circ.mcx([row6[0], row7[1], row8[2]], dg_result_4[4])

    circ.x(row7[1])

    circ.mcx([row6[0], row7[1], row8[2]], dg_result_4[4])

    circ.x(row6[0])
    circ.x(row7[1])
    circ.x(row8[2])

    # n=6
    circ.ccx(row7[0], row8[1], dg_result_4[5])
    circ.x(dg_result_4[5])

    return circ

In [ ]:
def phase_shift(circ, row1, row2, row3, row4, row5, row6, row7, row8,
                col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps):
    circ.mcx([col_result[0], col_result[1], col_result[2], col_result[3], col_result[4], col_result[5], col_result[6], col_result[7],
            dg_result_1[0], dg_result_1[1], dg_result_1[2], dg_result_1[3], dg_result_1[4], dg_result_1[5], dg_result_1[6],
            dg_result_2[0], dg_result_2[1], dg_result_2[2], dg_result_2[3], dg_result_2[4], dg_result_2[5],
            dg_result_3[0], dg_result_3[1], dg_result_3[2], dg_result_3[3], dg_result_3[4], dg_result_3[5], dg_result_3[6],
            dg_result_4[0], dg_result_4[1], dg_result_4[2], dg_result_4[3], dg_result_4[4], dg_result_4[5]],
            ps)

    return circ

In [ ]:
def oracle(circ, row1, row2, row3, row4, row5, row6, row7, row8,
          col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps):
    # column condition
    circ = col_cond(circ, row1, row2, row3, row4, row5, row6, row7, row8, col_result)
    # diagonal condition
    circ = dg_cond(circ, row1, row2, row3, row4, row5, row6, row7, row8,
                  dg_result_1, dg_result_2, dg_result_3, dg_result_4)
    # phase shift
    circ = phase_shift(circ, row1, row2, row3, row4, row5, row6, row7, row8,
                       col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps)
    # apply conditions once more to reset result qubits
    circ = col_cond(circ, row1, row2, row3, row4, row5, row6, row7, row8, col_result)
    circ = dg_cond(circ, row1, row2, row3, row4, row5, row6, row7, row8,
                  dg_result_1, dg_result_2, dg_result_3, dg_result_4)

    return circ

In [ ]:
def amplification(circ, row1, row2, row3, row4, row5, row6, row7, row8):
    # apply inverse of operator W
    circ = W_inv(circ, row1, row2, row3, row4, row5, row6, row7, row8)

    # apply X to every inputs
    for i in range(8):
        circ.x(row1[i])
        circ.x(row2[i])
        circ.x(row3[i])
        circ.x(row4[i])
        circ.x(row5[i])
        circ.x(row6[i])
        circ.x(row7[i])
        circ.x(row8[i])

    circ.h(row1[0])

    circ.mcx([row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7],
            row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7],
            row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7],
            row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7],
            row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7],
            row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7],
            row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7],
            row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]],
            row1[0])

    circ.h(row1[0])

    for i in range(8):
        circ.x(row1[i])
        circ.x(row2[i])
        circ.x(row3[i])
        circ.x(row4[i])
        circ.x(row5[i])
        circ.x(row6[i])
        circ.x(row7[i])
        circ.x(row8[i])

    circ = W(circ, row1, row2, row3, row4, row5, row6, row7, row8)

    return circ

In [ ]:
def num_operation(N, s):
    num = np.pi / 4 * np.sqrt(np.power(N, N) / s)
    return int(num)

In [ ]:
# main
# 8 rows of 8 quantum registers - total 64
row1 = QuantumRegister(8, name="row1")
row2 = QuantumRegister(8, name="row2")
row3 = QuantumRegister(8, name="row3")
row4 = QuantumRegister(8, name="row4")
row5 = QuantumRegister(8, name="row5")
row6 = QuantumRegister(8, name="row6")
row7 = QuantumRegister(8, name="row7")
row8 = QuantumRegister(8, name="row8")
# 8 quantum registers to store column condition result
col_result = QuantumRegister(8, name="col_result")
# 8 * 4 - 6 = 26 quantum registers to store diagonal condition result
dg_result_1 = QuantumRegister(7, name="dg_result_1")
dg_result_2 = QuantumRegister(6, name="dg_result_2")
dg_result_3 = QuantumRegister(7, name="dg_result_3")
dg_result_4 = QuantumRegister(6, name="dg_result_4")
# 1 quantum register for phase shift
ps = QuantumRegister(1, name="ps")
# 64 Classical Registers
c_row1 = ClassicalRegister(8, name="row1_measure")
c_row2 = ClassicalRegister(8, name="row2_measure")
c_row3 = ClassicalRegister(8, name="row3_measure")
c_row4 = ClassicalRegister(8, name="row4_measure")
c_row5 = ClassicalRegister(8, name="row5_measure")
c_row6 = ClassicalRegister(8, name="row6_measure")
c_row7 = ClassicalRegister(8, name="row7_measure")
c_row8 = ClassicalRegister(8, name="row8_measure")
# Circuit
circ = QuantumCircuit(row1, row2, row3, row4, row5, row6, row7, row8,
                      col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps,
                      c_row1, c_row2, c_row3, c_row4, c_row5, c_row6, c_row7, c_row8)

# uniform superposition
circ = initialize(circ, row1, row2, row3, row4, row5, row6, row7, row8, ps)

# number of operation, 8X8 chessboard with 92 solutions
num_op = num_operation(8, 92)
for _ in range(num_op):
    circ = oracle(circ, row1, row2, row3, row4, row5, row6, row7, row8,
          col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps)
    circ = amplification(circ, row1, row2, row3, row4, row5, row6, row7, row8)

circ.measure([row1[0], row1[1], row1[2], row1[3], row1[4], row1[5], row1[6], row1[7],
            row2[0], row2[1], row2[2], row2[3], row2[4], row2[5], row2[6], row2[7],
            row3[0], row3[1], row3[2], row3[3], row3[4], row3[5], row3[6], row3[7],
            row4[0], row4[1], row4[2], row4[3], row4[4], row4[5], row4[6], row4[7],
            row5[0], row5[1], row5[2], row5[3], row5[4], row5[5], row5[6], row5[7],
            row6[0], row6[1], row6[2], row6[3], row6[4], row6[5], row6[6], row6[7],
            row7[0], row7[1], row7[2], row7[3], row7[4], row7[5], row7[6], row7[7],
            row8[0], row8[1], row8[2], row8[3], row8[4], row8[5], row8[6], row8[7]],
            [c_row1[0], c_row1[1], c_row1[2], c_row1[3], c_row1[4], c_row1[5], c_row1[6], c_row1[7],
            c_row2[0], c_row2[1], c_row2[2], c_row2[3], c_row2[4], c_row2[5], c_row2[6], c_row2[7],
            c_row3[0], c_row3[1], c_row3[2], c_row3[3], c_row3[4], c_row3[5], c_row3[6], c_row3[7],
            c_row4[0], c_row4[1], c_row4[2], c_row4[3], c_row4[4], c_row4[5], c_row4[6], c_row4[7],
            c_row5[0], c_row5[1], c_row5[2], c_row5[3], c_row5[4], c_row5[5], c_row5[6], c_row5[7],
            c_row6[0], c_row6[1], c_row6[2], c_row6[3], c_row6[4], c_row6[5], c_row6[6], c_row6[7],
            c_row7[0], c_row7[1], c_row7[2], c_row7[3], c_row7[4], c_row7[5], c_row7[6], c_row7[7],
            c_row8[0], c_row8[1], c_row8[2], c_row8[3], c_row8[4], c_row8[5], c_row8[6], c_row8[7]])

#transpiled_circuit = transpile(circ, backend, optimization_level=3)
transpiled_circuit = cloud_transpiler_service.run(circ)
job = backend.run(transpiled_circuit)
results = job.result()
answer = results.get_counts()
sorted_answer = sorted(answer)
top_10 = dict(list(sorted_answer.items())[:10])

print(top_10)

Streaming output truncated to the last 5000 lines.
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 3.12328 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnrollCustomDefinitions - 0.21839 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('u', 1)} to target basis {'x', 'measure', 'reset', 'rx', 'ry', 'barrier', 'cx', 'p', 'rz', 'snapshot', 'u', 'delay', 'z'}.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation path search completed in 0.000s.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation paths composed in 0.000s.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation instructions replaced in 0.000s.
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 7.15899 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnrollCustomDefinitions - 0.23460 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('u', 1)} to target basis {'